In [113]:
%matplotlib inline
import matplotlib.pyplot as plt
import ujson as json
import glob as g
import random as rnd
import os.path as op
import os
import codecs
import shutil
import time
import math
import numpy as np
from skimage import io, transform
import skimage.color as skc

from tqdm import tqdm

import pandas as pd
import seaborn as sns

import multiprocessing
import hashlib as hl
import scipy.spatial.distance as sd
import tinydb as tb
import re

import subprocess as sp

In [45]:
def split_batch_name(batch):
    match = re.match('(\D+)(.*)',batch)
    return match.groups()
print split_batch_name('SB002/23')

('SB', '002/23')


In [59]:
tests_path = '/media/jakob/bigdata/line_comparision'
text_db_path = '/media/jakob/bigdata/train/db.json'
catalogs_dir = '/media/jakob/bigdata/train/'

In [123]:
data = pd.read_csv('/media/jakob/bigdata/train/sample_reproducibility.csv')['0']
sample_set = set(data)

In [124]:
def create_testbed(test_path, text_db_path,  ext = '.png'):
    db = tb.TinyDB(text_db_path)
    if not op.isdir(test_path):
        os.makedirs(test_path)
    samples = db.table('lines')
    for line in tqdm(samples.all()):
        if line['id'] in sample_set:
            cat_name, _ = split_batch_name(line['id'])
            cat_dir     = op.join(test_path,cat_name)
            if not op.isdir(cat_dir):
                os.makedirs(cat_dir)
            line_img = op.join(catalogs_dir, cat_name, line['id'])
            test_out = op.join(cat_dir, line['id'].replace('/', '_'))
            shutil.copy(line_img + ext, test_out + ext)
            with codecs.open(test_out + '.gt.txt', 'w', encoding='utf8') as gt_file:
                gt_file.write(line['gt'])
    db.close()


In [125]:
create_testbed(op.join(tests_path, 'tesseract'), text_db_path, ext='.png')
create_testbed(op.join(tests_path, 'ocropy_model_203000'), text_db_path, ext='.bin.png')
create_testbed(op.join(tests_path, 'ocropy_model_00101000'), text_db_path, ext='.bin.png')

100%|██████████| 335/335 [00:00<00:00, 13035.09it/s]


In [128]:
!ocropus-rpred -Q 5 -m /media/jakob/bigdata/train/model_2016_07_05/index_SB.model-00203000.pyrnn.gz '/media/jakob/bigdata/line_comparision/ocropy_model_203000/*/*.bin.png'
!ocropus-rpred -Q 5 -m /media/jakob/bigdata/train/model_2016_07_05/index_SB.model-00101000.pyrnn.gz '/media/jakob/bigdata/line_comparision/ocropy_model_00101000/*/*.bin.png'

INFO:  
INFO:  ########## /home/jakob/miniconda2/envs/htwmusik/bin/ocropus-rpred -Q 5 
INFO:  
INFO:  #inputs143
# loading object /media/jakob/bigdata/train/model_2016_07_05/index_SB.model-00203000.pyrnn.gz
INFO:  /media/jakob/bigdata/line_comparision/ocropy_model_203000/S/S014_00000001_010008.bin.png:No. 2 -
INFO:  /media/jakob/bigdata/line_comparision/ocropy_model_203000/S/S007_00000726_010004.bin.png:itn Hie, O scar l).:s Klavier uu. s Mleist,:r.
INFO:  /media/jakob/bigdata/line_comparision/ocropy_model_203000/S/S021_00000369_010004.bin.png:AAul euS. iSd7guus. )
INFO:  /media/jakob/bigdata/line_comparision/ocropy_model_203000/S/S022_00000090_010001.bin.png:AH -r, 1.F 3.TC..
INFO:  /media/jakob/bigdata/line_comparision/ocropy_model_203000/S/S001_00000051_010005.bin.png:Aillst du haban mici: nur um Singen,' a u
INFO:  /media/jakob/bigdata/line_comparision/ocropy_model_203000/S/S007_00000726_010002.bin.png:Albert, E ugen d Studie iüb. d. lraumi der Ghismionda a. d. Operühis.
INFO:  /me

In [129]:
img_paths = g.glob('/media/jakob/bigdata/line_comparision/tesseract/*/*.png')
for path in tqdm(img_paths):
    base = op.splitext(path)[0]
    call_output = sp.check_output(["tesseract -psm 7 -l deu {} {}".format(path, base)], shell=True, stderr=sp.STDOUT)

100%|██████████| 143/143 [01:14<00:00,  1.84it/s]


In [109]:
!cat /media/jakob/bigdata/line_comparision/tesseract/SC/SC019_00000400_010007.txt

Emh.: Suite Violoncello solo op. 84. Dyophonie op. 24]. Konzert für Violoncello und



In [127]:
!ls /media/jakob/bigdata/line_comparision/tesseract/SC/

SC002_00000491_010003.gt.txt  SC019_00000350_010009.txt
SC002_00000491_010003.png     SC019_00000400_010007.gt.txt
SC002_00000491_010003.txt     SC019_00000400_010007.png
SC003_00000842_010004.gt.txt  SC019_00000400_010007.png.txt
SC003_00000842_010004.png     SC019_00000400_010007.txt
SC003_00000842_010004.txt     SC024_00000555_010008.gt.txt
SC004_00000418_01000c.gt.txt  SC024_00000555_010008.png
SC004_00000418_01000c.png     SC024_00000555_010008.txt
SC004_00000418_01000c.txt     SC024_00000697_01000a.gt.txt
SC006_00000941_010004.gt.txt  SC024_00000697_01000a.png
SC006_00000941_010004.png     SC024_00000697_01000a.txt
SC006_00000941_010004.txt     SC024_00000744_010006.gt.txt
SC012_00000002_010003.gt.txt  SC024_00000744_010006.png
SC012_00000002_010003.png     SC024_00000744_010006.txt
SC012_00000002_010003.txt     SC027_00000908_010008.gt.txt
SC012_00000630_010006.gt.txt  SC027_00000908_010008.png
SC012_00000630_010006.png     SC027_00000908_010008.txt
SC012_00000630_010006.txt    

In [126]:
len(img_paths)

115